In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022826194763183594
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 14.890477164343656
200000 25.04476255014836
300000 24.837975636091638
400000 24.63440335972296
500000 26.079251887837472
600000 25.618282540458882
700000 23.779790376660895
800000 24.08884622456712
900000 22.953982794701894
1000000 10.393853620079488
1100000 23.7589839489424
1200000 23.8325373258402
1300000 23.524140424228907
1400000 24.778435556213072
1500000 24.536107337826426
1600000 24.084773451587093
1700000 24.2470631168275
1800000 24.36604986797461
1900000 9.824107762485982
2000000 24.90554867625355
2100000 29.541254022125692
2200000 28.71828408963136
2300000 29.007514826268213
2400000 29.53005482202458
2500000 27.784539041590154
2600000 27.577613424194894
2700000 27.743738056989592
2800000 17.380304899231195
2900000 21.834620392344306
3000000 27.

29400000 23.755138292498025
29500000 23.61020897872577
29600000 11.94552073460845
29700000 23.41092047585754
29800000 23.933887171367942
29900000 23.791998929198805
30000000 21.473654519393673
30100000 12.690414588599284
30200000 24.341751422063354
30300000 24.116809275394463
30400000 23.35661287257056
30500000 22.86291227080608
30600000 21.262373528728247
30700000 16.37249062805003
30800000 15.010621027910783
30900000 21.367655370495356
31000000 22.477001187382182
31100000 23.27185534669203
31200000 23.37508793414773
31300000 24.412994926897465
31400000 15.179759009904288
31500000 20.91685528572853
31600000 23.956342095400128
31700000 25.01385821126725
31800000 24.18021310035238
31900000 17.821825435591172
32000000 18.519250248051282
32100000 24.896498620651602
32200000 23.568905688680918
32300000 24.186206763982934
32400000 22.116445098606615
32500000 21.296605018804907
32600000 8.810883963539244
32700000 22.383657884515724
32800000 21.476125790115702
32900000 23.467125619884495
3300

59100000 10.883135233375736
59200000 19.671450067009975
59300000 21.714796781101256
59400000 20.88217627297129
59500000 22.874125375986818
59600000 21.686616564321472
59700000 22.008267984898378
59800000 21.289944489993033
59900000 19.959315837010855
60000000 22.227389581900727
60100000 8.941302075446822
60200000 18.394825587187558
60300000 20.03875098377757
60400000 18.57324039633748
60500000 9.575377177663466
60600000 18.40732692054733
60700000 18.7367164951927
60800000 19.54994992751505
60900000 16.498015762678875
61000000 13.265164790314703
61100000 20.99123323107548
61200000 20.17515795092237
61300000 20.34501421891558
61400000 21.689759240477738
61500000 21.259076131464923
61600000 22.540796389479787
61700000 20.9448309424199
61800000 21.572829634759813
61900000 16.881853493558804
62000000 11.87522575596975
62100000 20.17438379561898
62200000 19.49929396746382
62300000 20.50655285100418
62400000 7.810513442795621
62500000 18.905009617854297
62600000 19.210113493888084
62700000 19

88900000 17.686382074346362
89000000 17.510161773928175
89100000 17.74416703940109
89200000 18.75343754239619
89300000 19.197019553447202
89400000 19.588645205232602
89500000 18.982186365473144
89600000 18.278500052738053
89700000 17.14151480627147
89800000 17.243802061130136
89900000 17.803905448621975
90000000 16.919041066179265
90100000 8.481003269435167
90200000 16.86253929990347
90300000 6.508942990868284
90400000 17.524566132099896
90500000 8.97634109008345
90600000 17.391474190004722
90700000 17.55559201433028
90800000 17.35368518655667
90900000 17.249616219073506
91000000 17.828813945642082
91100000 18.904311897786254
91200000 18.979102560144547
91300000 20.004272322499315
91400000 18.666823778949826
91500000 17.33033576641856
91600000 17.095824322837174
91700000 17.606389466064066
91800000 17.416762277778133
91900000 16.810747669502106
92000000 8.772553041597702
92100000 13.865916226307114
92200000 10.647389665819176
92300000 11.966798152023319
92400000 13.873362417001326
9250

117900000 1.6940030773330694
118000000 1.6914475337651076
118100000 1.693795550722503
118200000 1.7001726221766174
118300000 1.7172142971733508
118400000 1.7230628987459542
118500000 1.746488744019028
118600000 1.7461768525124888
118700000 1.7373890649878085
118800000 1.7451748079714677
118900000 1.698231312438965
119000000 1.6918740729287234
119100000 2.5208726352581845
119200000 22.43933846707941
119300000 20.963596682450394
119400000 20.855077056495613
119500000 20.369889039521535
119600000 20.318081372103347
119700000 19.945796964115626
119800000 19.486270781985613
119900000 20.156252773538174
120000000 10.449704495055366
120100000 2.6024742231639424
120200000 18.355622817635318
120300000 19.33750797763693
120400000 20.088616476322066
120500000 19.760405462522492
120600000 20.94489036035176
120700000 20.223105493272094
120800000 21.00037552857237
120900000 21.530487527265926
121000000 20.744431494930257
121100000 17.34101627675661
121200000 17.11795712329487
121300000 16.0390370693

...

232900000 2.163819931802094
233000000 2.165147483977364
233100000 2.2249489859338967
233200000 2.9340135281267763
233300000 1.9855065583022733
233400000 2.202250996051497
233500000 0.9494860798395574
233600000 0.5267412942407496
233700000 1.486968536131147
233800000 1.9998789871190983
233900000 1.9698262615652913
234000000 2.0689122819076116
234100000 2.0591121065715554
234200000 2.0139511021523244
234300000 2.0115208042508463
234400000 1.8636596164746682
234500000 2.20786676285348
234600000 2.2436566914097136
234700000 1.8631473455907803
234800000 1.999755032310264
234900000 2.022224627733184
235000000 1.9990570519165995
235100000 2.0562771706597243
235200000 2.5415305996357174
235300000 2.1702744364733992
235400000 0.8763780623917414
235500000 0.5350342892114851
235600000 2.5432224583981538
235700000 2.056804444892969
235800000 2.6777078999148705
235900000 1.9521028998872338
236000000 2.201476211469942
236100000 2.0373746728772884
236200000 1.8747175240462521
236300000 2.039357395439

261600000 1.758673088770734
261700000 2.9911329924400745
261800000 3.247953157805969
261900000 3.180753902799027
262000000 3.0331046288709342
262100000 3.079267844966458
262200000 3.0595077464514278
262300000 3.0742668392695784
262400000 4.006267967621064
262500000 3.839797134798212
262600000 3.4223407990916273
262700000 3.147680531366471
262800000 3.2423316612779898
262900000 3.269102232719779
263000000 3.2819201395290696
263100000 3.0485817890575824
263200000 3.5290029078571696
263300000 0.759357825356288
263400000 0.7629742830763161
263500000 2.8607403428728277
263600000 3.260244844264123
263700000 3.2452448250557815
263800000 3.4837685039675836
263900000 3.261860777780377
264000000 2.9404032504645916
264100000 3.2648937665200037
264200000 3.3160734983459905
264300000 4.51929179069979
264400000 4.061435568067779
264500000 3.663038341132025
264600000 3.3677347907395463
264700000 3.720672180463493
264800000 3.7737245737230283
264900000 3.494953296918932
265000000 3.144215145420005
265

290300000 2.562656729580621
290400000 2.415441151704328
290500000 2.0722023857339025
290600000 2.0608831480336685
290700000 2.2346201642956234
290800000 2.193933092707157
290900000 1.9871365234260063
291000000 1.982000404508319
291100000 2.138661200099081
291200000 1.0685196860686474
291300000 0.5935538398593034
291400000 1.8075142160311342
291500000 1.8586731655931141
291600000 2.030601291388485
291700000 2.0900424421562853
291800000 2.417812164432032
291900000 1.9437874907242192
292000000 2.1031028789212107
292100000 2.1761462900461357
292200000 2.8101340989708765
292300000 2.1178979597956538
292400000 2.1276480857013063
292500000 1.8438784912552968
292600000 2.0004521191037
292700000 2.3527252828747263
292800000 2.143462210391078
292900000 1.9340005186580418
293000000 1.6899890520576701
293100000 0.47589162634583976
293200000 0.4776592666831314
293300000 2.6522289266376027
293400000 1.938242761245115
293500000 2.0170663272857667
293600000 1.9968695747598444
293700000 2.3128604320092

318100000 1.1754881578582923
318200000 1.140811873227785
318300000 1.2505152752002477
318400000 1.4405445671168269
318500000 1.1266964446783463
318600000 1.3589676599462797
318700000 1.2117099562904239
318800000 1.399146703307738
318900000 1.2416391934761835
319000000 1.1833075981829564
319100000 1.1170917129322953
319200000 1.500687584161745
319300000 1.236074277834743
319400000 1.1697381321305533
319500000 1.2186228456888595
319600000 1.2125985279932023
319700000 1.1172050173929367
319800000 1.3931307737760013
319900000 1.1002820470575094
320000000 1.4377321780519516
320100000 1.2208181839127574
320200000 1.1631899333253006
320300000 1.1659554582385587
320400000 1.2156291448948382
320500000 1.20505075837183
320600000 1.145801272047046
320700000 1.1459162352588574
320800000 1.2065604149068496
320900000 1.0695650035276116
321000000 1.1613138764935433
321100000 1.145626595782419
321200000 1.1100403377226558
321300000 1.0500360445252492
321400000 1.0426677833510638
321500000 1.0018831191

345100000 0.23873761640758315
345200000 0.17207711190962463
345300000 0.23377332002530496
345400000 0.1650467721150021
345500000 0.22405354012981718
345600000 0.17097086372359263
345700000 0.18818864299786092
345800000 0.17448136058582864
345900000 0.1692158507038719
346000000 0.17054454496424398
346100000 0.14405172278682393
346200000 0.18420341554269196
346300000 0.15197766705299418
346400000 0.19460029735975798
346500000 0.14884411874968476
346600000 0.15463746757486463
346700000 0.1803887859869136
346800000 0.13465838474053807
346900000 0.14328752150830626
347000000 0.12256032780562176
347100000 0.15658528351834416
347200000 0.13262906722643972
347300000 0.13424873553684682
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.08002346463633908
347500000 0.01919175724697113
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of